In [15]:
import os
from glob import glob
from utils import compute_mean_dice
import nibabel as nib
from scipy.spatial.distance import dice
import numpy as np
import itk
import SimpleITK as sitk
import scipy.ndimage
import scipy
import matplotlib.pyplot as plt
from transforms_dict import getRegistrationEvalInverseTransformForMRI, SaveTransformForMRI
from tqdm import tqdm
import monai
import subprocess
from monai.networks.blocks import Warp
import torch
import pandas as pd

In [8]:
def query_data_by_name(data, name):
    list = []
    for i in range(len(data)):
        if data['acronym'][i] == name:
            j = i 
            id = data['id'][j]
    id = "/" + str(id) + "/"
    for i in range(len(data)):
        if id in data['structure_id_path'][i]:
            list.append(data['graph_order'][i])
    return list    

In [14]:
allen_csv = "dataset3/Atlas/query.csv"  
allen_data = pd.read_csv(allen_csv)
atlas_labels = "dataset3/Atlas/P56_Annotation_128_norm_id.nii.gz"
codes = ["fiber tracts", "grey", "VS"]
values = [2,1,3]
label_load = nib.load(atlas_labels)
image_zero = np.zeros(label_load.shape)
for i, code in enumerate(codes):    
    liste = query_data_by_name(allen_data, codes[i])        
    image_zero[np.where(np.isin(label_load.get_fdata(), liste))] = values[i]
image = nib.Nifti1Image(image_zero, label_load.affine, label_load.header)
nib.save(image, "dataset3/Atlas/atlas_gmwmlcr.nii.gz")

In [20]:
ptdr = nib.load("dataset3/Atlas/c1c2c3_P56_0.01.nii.gz")
ptdr = ptdr.get_fdata()
x1 = (ptdr==1).sum()
print(x1)
x2 = (ptdr==2).sum()
print(x2)
x3 = (ptdr==3).sum()
print(x3)
tot = x1+x2+x3
print(x1/tot*100)
print(x2/tot*100)
print(x3/tot*100)

244843
80491
24572
69.97393585705876
23.0036066829377
7.022457460003544


In [23]:
wm_labels = "dataset3/Atlas/white_P56_id.nii.gz"
gm_labels = "dataset3/Atlas/grey_P56_id.nii.gz"
cs_labels = "dataset3/Atlas/csf_P56_id.nii.gz"



mris = sorted(glob(os.path.join('dataset3', 'Femina3', 'MRI', '*_id.nii.gz')))
affine_fwd_warps = sorted(glob(os.path.join('dataset3', 'Femina3', 'MRI', '*.mat')))
mean_fwd_warps =   sorted(glob(os.path.join('dataset3', 'Femina3', 'MRI', "*_warp.nii.gz")))
mean_bck_warps =   sorted(glob(os.path.join('dataset3', 'Femina3', 'MRI', "*_warp_inverse.nii.gz")))

for i in range(len(affine_fwd_warps)):       
    output = "dataset3/Femina3/LabelsBackFromAtlas/" + "_".join(affine_fwd_warps[i].split('/')[-1].split('.')[0].split('_')) + '_fromsyn_spm_c1.nii.gz'
    output = output.replace("_affine_","_")
    command = ['antsApplyTransforms']
    command.extend(['-d', '3'])
    command.extend(['-i', gm_labels])
    command.extend(['-r', mris[i]])
    command.extend(['-o', output])
    command.extend(['-n', 'Linear'])
    command.extend(['-t', '[' + affine_fwd_warps[i] + ',1]'])
    command.extend(['-t', mean_bck_warps[i]])
    subprocess.call(command) 
    c1 = nib.load(output).get_fdata()
    
    output = "dataset3/Femina3/LabelsBackFromAtlas/" + "_".join(affine_fwd_warps[i].split('/')[-1].split('.')[0].split('_')) + '_fromsyn_spm_c2.nii.gz'
    output = output.replace("_affine_","_")
    command = ['antsApplyTransforms']
    command.extend(['-d', '3'])
    command.extend(['-i', wm_labels])
    command.extend(['-r', mris[i]])
    command.extend(['-o', output])
    command.extend(['-n', 'Linear'])
    command.extend(['-t', '[' + affine_fwd_warps[i] + ',1]'])
    command.extend(['-t', mean_bck_warps[i]])
    subprocess.call(command) 
    c2 = nib.load(output).get_fdata()
    
    output = "dataset3/Femina3/LabelsBackFromAtlas/" + "_".join(affine_fwd_warps[i].split('/')[-1].split('.')[0].split('_')) + '_fromsyn_spm_c3.nii.gz'
    output = output.replace("_affine_","_")
    command = ['antsApplyTransforms']
    command.extend(['-d', '3'])
    command.extend(['-i', cs_labels])
    command.extend(['-r', mris[i]])
    command.extend(['-o', output])
    command.extend(['-n', 'Linear'])
    command.extend(['-t', '[' + affine_fwd_warps[i] + ',1]'])
    command.extend(['-t', mean_bck_warps[i]])
    subprocess.call(command) 
    c3 = nib.load(output).get_fdata()
    
    xdqds = nib.load(mris[i])
    affine = xdqds.affine
    header = xdqds.header    
    output = "dataset3/Femina3/LabelsBackFromAtlas/" + "_".join(affine_fwd_warps[i].split('/')[-1].split('.')[0].split('_')) + '_fromsyn_spm_c1c2c3.nii.gz'
    output = output.replace("_affine_","_")
    tab = np.argmax((0.1*np.ones(c1.shape), c1, c2, c3), 0)
    img = nib.Nifti1Image(tab, affine, header)
    nib.save(img, output)

In [24]:
wm_labels = "dataset3/Atlas/white_P56_id.nii.gz"
gm_labels = "dataset3/Atlas/grey_P56_id.nii.gz"
cs_labels = "dataset3/Atlas/csf_P56_id.nii.gz"

mris = sorted(glob(os.path.join('dataset3', 'Femina3', 'MRI', '*_id.nii.gz')))
affine_fwd_warps = sorted(glob(os.path.join('dataset3', 'Femina3', 'MRI', '*.mat')))
mean_fwd_warps =   sorted(glob(os.path.join('output', 'Femina3', 'test', 'DeformableWarp', "*_warp.nii.gz")))
mean_bck_warps =   sorted(glob(os.path.join('output', 'Femina3', 'test', 'DeformableWarp', "*_invwarp.nii.gz")))

for i in range(len(affine_fwd_warps)):       
    output = "dataset3/Femina3/LabelsBackFromAtlas/" + "_".join(affine_fwd_warps[i].split('/')[-1].split('.')[0].split('_')) + '_fromop_spm_c1.nii.gz'
    output = output.replace("_affine_","_")
    command = ['antsApplyTransforms']
    command.extend(['-d', '3'])
    command.extend(['-i', gm_labels])
    command.extend(['-r', mris[i]])
    command.extend(['-o', output])
    command.extend(['-n', 'Linear'])
    command.extend(['-t', '[' + affine_fwd_warps[i] + ',1]'])
    command.extend(['-t', mean_bck_warps[i]])
    subprocess.call(command) 
    c1 = nib.load(output).get_fdata()
    
    output = "dataset3/Femina3/LabelsBackFromAtlas/" + "_".join(affine_fwd_warps[i].split('/')[-1].split('.')[0].split('_')) + '_fromop_spm_c2.nii.gz'
    output = output.replace("_affine_","_")
    command = ['antsApplyTransforms']
    command.extend(['-d', '3'])
    command.extend(['-i', wm_labels])
    command.extend(['-r', mris[i]])
    command.extend(['-o', output])
    command.extend(['-n', 'Linear'])
    command.extend(['-t', '[' + affine_fwd_warps[i] + ',1]'])
    command.extend(['-t', mean_bck_warps[i]])
    subprocess.call(command) 
    c2 = nib.load(output).get_fdata()
    
    output = "dataset3/Femina3/LabelsBackFromAtlas/" + "_".join(affine_fwd_warps[i].split('/')[-1].split('.')[0].split('_')) + '_fromop_spm_c3.nii.gz'
    output = output.replace("_affine_","_")
    command = ['antsApplyTransforms']
    command.extend(['-d', '3'])
    command.extend(['-i', cs_labels])
    command.extend(['-r', mris[i]])
    command.extend(['-o', output])
    command.extend(['-n', 'Linear'])
    command.extend(['-t', '[' + affine_fwd_warps[i] + ',1]'])
    command.extend(['-t', mean_bck_warps[i]])
    subprocess.call(command) 
    c3 = nib.load(output).get_fdata()
    
    xdqds = nib.load(mris[i])
    affine = xdqds.affine
    header = xdqds.header    
    output = "dataset3/Femina3/LabelsBackFromAtlas/" + "_".join(affine_fwd_warps[i].split('/')[-1].split('.')[0].split('_')) + '_fromop_spm_c1c2c3.nii.gz'
    output = output.replace("_affine_","_")
    tab = np.argmax((0.1*np.ones(c1.shape), c1, c2, c3), 0)
    img = nib.Nifti1Image(tab, affine, header)
    nib.save(img, output)

In [8]:
wmgmlcr_labels = "dataset3/Atlas/c1c2c3_P56.nii.gz"
affine_fwd_warps = sorted(glob(os.path.join('dataset3', 'GIN', 'MRI', '*.mat')))
mean_fwd_warps = sorted(glob(os.path.join('output', 'GIN', 'ith_lambda2_0', 'DeformableWarp', "*_warp.nii.gz")))
mean_bck_warps = sorted(glob(os.path.join('output', 'GIN', 'ith_lambda2_0', 'DeformableWarp', "*_invwarp.nii.gz")))

for i in range(len(affine_fwd_warps)):        
    output = "dataset3/GIN/LabelsBackFromAtlas/" + affine_fwd_warps[i].split('/')[-1].split('_')[0] + '_id_frompair.nii.gz'
    command = ['antsApplyTransforms']
    command.extend(['-d', '3'])
    command.extend(['-i', wmgmlcr_labels])
    command.extend(['-r', wmgmlcr_labels])
    command.extend(['-o', output])
    command.extend(['-n', 'MultiLabel'])
    command.extend(['-t', '[' + affine_fwd_warps[i] + ',1]'])
    command.extend(['-t', mean_bck_warps[i]])
    subprocess.call(command) 

In [10]:
wmgmlcr_labels = "dataset3/Atlas/atlas_wmgmlcr.nii.gz"
affine_fwd_warps = sorted(glob(os.path.join('dataset3', 'GIN', 'MRI', '*.mat')))
mean_fwd_warps = sorted(glob(os.path.join('output', 'GIN', 'lambda2_100', 'DeformableWarp', "*_warp.nii.gz")))
mean_bck_warps = sorted(glob(os.path.join('output', 'GIN', 'lambda2_100', 'DeformableWarp', "*_invwarp.nii.gz")))

for i in range(len(affine_fwd_warps)):        
    output = "dataset3/GIN/LabelsBackFromAtlas/" + affine_fwd_warps[i].split('/')[-1].split('_')[0] + '_id_fromop2.nii.gz'
    command = ['antsApplyTransforms']
    command.extend(['-d', '3'])
    command.extend(['-i', wmgmlcr_labels])
    command.extend(['-r', wmgmlcr_labels])
    command.extend(['-o', output])
    command.extend(['-n', 'MultiLabel'])
    command.extend(['-t', '[' + affine_fwd_warps[i] + ',1]'])
    command.extend(['-t', mean_bck_warps[i]])
    subprocess.call(command) 

In [11]:
wmgmlcr_labels = "dataset3/Atlas/atlas_wmgmlcr.nii.gz"
affine_fwd_warps = sorted(glob(os.path.join('dataset3', 'GIN', 'MRI', '*.mat')))
mean_fwd_warps = sorted(glob(os.path.join('output', 'GIN', 'ith_lambda2_2', 'DeformableWarp', "*_warp.nii.gz")))
mean_bck_warps = sorted(glob(os.path.join('output', 'GIN', 'ith_lambda2_2', 'DeformableWarp', "*_invwarp.nii.gz")))

for i in range(len(affine_fwd_warps)):        
    output = "dataset3/GIN/LabelsBackFromAtlas/" + affine_fwd_warps[i].split('/')[-1].split('_')[0] + '_id_frompair2.nii.gz'
    command = ['antsApplyTransforms']
    command.extend(['-d', '3'])
    command.extend(['-i', wmgmlcr_labels])
    command.extend(['-r', wmgmlcr_labels])
    command.extend(['-o', output])
    command.extend(['-n', 'MultiLabel'])
    command.extend(['-t', '[' + affine_fwd_warps[i] + ',1]'])
    command.extend(['-t', mean_bck_warps[i]])
    subprocess.call(command) 

In [12]:
wmgmlcr_labels = "dataset3/Atlas/atlas_wmgmlcr.nii.gz"
affine_fwd_warps = sorted(glob(os.path.join('dataset3', 'GIN', 'MRI', '*.mat')))
mean_fwd_warps = sorted(glob(os.path.join('output', 'GIN', 'dl-iris-sym', 'DeformableWarp', "*_warp.nii.gz")))
mean_bck_warps = sorted(glob(os.path.join('output', 'GIN', 'dl-iris-sym', 'DeformableWarp', "*_invwarp.nii.gz")))

for i in range(len(affine_fwd_warps)):        
    output = "dataset3/GIN/LabelsBackFromAtlas/" + affine_fwd_warps[i].split('/')[-1].split('_')[0] + '_id_fromdl.nii.gz'
    command = ['antsApplyTransforms']
    command.extend(['-d', '3'])
    command.extend(['-i', wmgmlcr_labels])
    command.extend(['-r', wmgmlcr_labels])
    command.extend(['-o', output])
    command.extend(['-n', 'MultiLabel'])
    command.extend(['-t', '[' + affine_fwd_warps[i] + ',1]'])
    command.extend(['-t', mean_bck_warps[i]])
    subprocess.call(command) 

In [ ]:
ants_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'LabelsBackFromAtlas', '*_id.nii.gz')))ants_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'LabelsBackFromAtlas', '*_id.nii.gz')))